In [1]:
from configspark import create_session, read_data
import pyspark.sql.functions as f 
import sparknlp
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from pyspark.ml.feature import HashingTF, IDF, StringIndexer, SQLTransformer, IndexToString
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

## Goal
* Objective is to classify Reviews based on ratings to be honest you probably need to do chisquared first to see uniqueness of words per class
  but that doesn't stop the fun this project is to show how to pipeline the data and some basic cleaning not trying to get the best model 
* The note books are separated in parts because I want to exime inside the pipelines.

##### Configspark
* I was being lazy and I added the spark config function and also the read data it's all in the configspark.py at list I added a schema 

In [2]:
spark = create_session()

:: loading settings :: url = jar:file:/Users/tonderaimadamba/spark_nlp_classification/spark_env/lib/python3.11/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/tonderaimadamba/.ivy2/cache
The jars for the packages stored in: /Users/tonderaimadamba/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-39a81e3d-8afa-46b1-8a45-09ec4b255f92;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.0.2 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in central
	found com

In [3]:
df = read_data(spark)

##### Split the data to train and validate 
* when you fit a pipeline I line to transform a different dataset than the one I have 

##### Split the data to train and validate 
* when you fit a pipeline I line to transform a different dataset than the one I have 

In [4]:
train_ratio = 0.7
test_ratio = 0.15
validation_ratio = 0.15

# Split the data using randomSplit()
train_data, test_data, validation_data = df.randomSplit([train_ratio, test_ratio, validation_ratio], seed=45)


In [5]:
train_data

DataFrame[review: string, rating: string]

#### Pipelining
* like it says pipeline it's a chain the previous transformation is linked to the next transformation order of excecution is maintained 

In [6]:
# Preprocess the data to concatenate feature columns into one column called text
# featureConcat = FeatureConcatenator(outputCols = ["text"], inputCols = [target_col])

# Prepares data into a format that is processable by Spark NLP. This is the entry point for every Spark NLP pipeline. 
# The DocumentAssembler can read either a String column or an Array[String]
documentAssembler = DocumentAssembler().setInputCol("review").setOutputCol("document")

# Tokenizes raw text in document type columns into TokenizedSentence
tokenizer = Tokenizer().setInputCols("document").setOutputCol("token")

# Annotator that cleans out tokens.
# Remove white space
normalizer = Normalizer().setInputCols("token").setOutputCol("normalized").setLowercase(True).setCleanupPatterns(["[^\w\s]"])

# Remove years (integers starting with 19XX or 20XX)
removeYear = Normalizer().setInputCols(["normalized"]).setOutputCol("remove_year").setCleanupPatterns(["(?:(?:19|20)\d\d)"])

# Find lemmas out of words with the objective of returning a base dictionary word
lemmatizer = LemmatizerModel.pretrained().setInputCols("remove_year").setOutputCol("lemmatized")

# A feature transformer that converts the input array of strings (annotatorType TOKEN) into an array of n-grams (annotatorType CHUNK). 
#  Null values in the input array are ignored. It returns an array of n-grams where each n-gram is represented by a space-separated string of words.
ngrammer = NGramGenerator().setInputCols(['lemmatized']).setOutputCol('ngrams').setN(3).setEnableCumulative(True).setDelimiter('_')

# Converts annotation results into a format that easier to use. It is useful to extract the results from Spark NLP Pipelines. 
# The Finisher outputs annotation(s) values into String
finisher = Finisher().setInputCols(['ngrams']).setOutputCols(['final'])

# Maps a sequence of terms to their term frequencies using the hashing trick.
tfizer= HashingTF(inputCol='final', outputCol='tf_features')

# Inverse document frequency
# This implementation supports filtering out terms which do not appear in a minimum number of documents
# idf = log((m + 1) / (d(t) + 1)), where m is the total number of documents and d(t) is the number of documents that contain term t.
# the number of documents is the number of classes 
idfizer = IDF(inputCol='tf_features', outputCol="features", minDocFreq = 2)

stringIndexer = StringIndexer(inputCol = "rating", outputCol = "label").fit(train_data)
inverter = IndexToString(inputCol = "prediction", outputCol = "predictionLabel", labels = stringIndexer.labels)


lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ]lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
Download done! Loading the resource.
[OK!]


In [7]:
stringIndexer = StringIndexer(inputCol = "rating", outputCol = "label").fit(train_data)

In [8]:
transformed_Labels = stringIndexer.transform(train_data)

In [9]:
transformed_Labels.show() ## The unique labels are just labels they mean nothing

+--------------------+------+-----+
|              review|rating|label|
+--------------------+------+-----+
|"5 Stars - ""Very...|   5.0|  0.0|
|"ALMOST everythin...|   3.0|  4.0|
|"After a few week...|   4.0|  2.0|
|"After my old Ham...|   1.0|  1.0|
|"After years of u...|   5.0|  0.0|
|"As a coffee novi...|   5.0|  0.0|
|"At first, I like...|   2.0|  3.0|
|"Best coffee pot ...|   5.0|  0.0|
|"Calling this thi...|   1.0|  1.0|
|"Coffee maker lea...|   1.0|  1.0|
|"Coffee pot came ...|   1.0|  1.0|
|"Concerns with th...|   2.0|  3.0|
|"Easy and classic...|   5.0|  0.0|
|"Ever part of it ...|   1.0|  1.0|
|"Excellent coffee...|   5.0|  0.0|
|"Excellent produc...|   5.0|  0.0|
|"FIRST IMPRESSION...|   3.0|  4.0|
|"FULL DISCLOSURE:...|   5.0|  0.0|
|"First, understan...|   2.0|  3.0|
|"First-Nearly 6- ...|   5.0|  0.0|
+--------------------+------+-----+
only showing top 20 rows

